In [ ]:
%%time
print("starting")
from checkra import happyscribe_scrape as happy

main = "https://www.happyscribe.com"                
lexurl="/public/lex-fridman-podcast-artificial-intelligence-ai"
billionaireurl = "/public/we-study-billionaires-the-investor-s-podcast-network"
lexhtml_location = "happyscribe_Lex_Fridman_html.txt"
billhtml_location="happyscribe_billionaires_podcast.txt"
happy.update_transcripts(billhtml_location, main)

In [ ]:
%%time

import spacy
import os
from os import listdir
from os.path import isfile, join
from tqdm import tqdm
import re
from checkra.nlp_pipeline import nlp, run_pipeline
from checkra.text_clean import text_fix, strip_accents
doc = nlp(text_fix(open("data/We_Study_Billionaires_-_The_Investor’s_Podcast_Network/TIP325|Real_Estate_Investing_with_BiggerPockets_(Business_Podcast).txt").read()))
# doc = nlp(text_fix(open("3Lex/#136|Dan_Carlin|Hardcore_History.txt").read()))
# print(doc.user_data["stamps"])
# print(doc.user_data.keys())

In [ ]:
docs = run_pipeline("data/3Lex", "Lex Fridman")

In [ ]:
docs1 = run_pipeline("data/8Lex", "Lex Fridman")

In [ ]:
docs2 = run_pipeline("data/We_Study_Billionaires_-_The_Investor’s_Podcast_Network", "Investors Podcast")

In [ ]:
for ind, doc in enumerate(docs2):
    print(ind, doc.user_data["guest"])

In [ ]:
docs[0].user_data["traits"].keys()

In [ ]:
import shutil
count = 0
not_processed = []
for ind, file in enumerate(os.listdir("7Lex")):
    if file.endswith(".txt"):
        file2 = file.replace("_", " ").replace("|", " ").replace(".txt", " ")
    #     print(ind, file) 
        processed = False


        for doc in docs:
            if doc.user_data["guest"] in file2 and doc.user_data["title"] in file2:
    #             os.rename("5Lex/"+file, "7Lex/"+file)
                
                processed = True
                count+=1
                break
        if processed:
            os.rename("7Lex/"+file, "8Lex/"+file)
print("done")

In [ ]:
import pymongo
from pymongo import MongoClient
import urllib

username = urllib.parse.quote_plus('Empyr3an')
password = urllib.parse.quote_plus("B4ldr1c7@1")

cluster = pymongo.MongoClient("mongodb+srv://{}:{}@cluster0.4pec2.mongodb.net/Checkra?retryWrites=true&w=majority".format(username, password))
db = cluster["production"]

# db.create_collection("lmini")

In [ ]:
db["Lex Fridman Podcast"].drop()

db["Lex Fridman Podcast"].insert_many([doc.user_data for doc in docs])
db["Lex Fridman Podcast"].insert_many([doc.user_data for doc in docs1])

In [ ]:
db["The Investor's Podcast"].drop()
db["The Investor's Podcast"].insert_many([doc.user_data for doc in docs2])

In [ ]:
all_atts = []
for doc in list(collection.find({"books":{"$ne":[]}})):
    print(doc["guest"], doc[att])
    for a in doc[att]:
        all_atts.append(a)

In [ ]:
%%time
#loading in previous bytestreams, needs 2 since 2 podcast folders edit later
doc_bin = DocBin(store_user_data=True) #docbin container for serialization

    doc_bin.add(doc) #add doc to list and bin
    
# with open("cucumber/Lexnumbered.bin", "wb") as f: #write bytestream to first_doc.bin
#     f.write(doc_bin.to_bytes())

with open("cucumber/Lex.bin", "rb") as w: 
    new_docs1 = DocBin(store_user_data=True).from_bytes(w.read())
docs1 = list(new_docs1.get_docs(nlp.vocab))